In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cv2
import json
import os
import datetime
import nrrd
import bg_space as bg
from pprint import pprint
from matplotlib.colors import LogNorm, Normalize
import seaborn as sb
import pathlib
import pathlib
from bg_atlasapi import BrainGlobeAtlas
from lcm_registration import LCM_registration_functions as lrf
# from LCM_registration_functions import get_z_value
# from LCM_registration_functions import get_roi_vol
from lcm_registration import visualign_functions as vis
from znamutils import slurm_it


In [ ]:
bg_atlas = BrainGlobeAtlas("allen_mouse_25um", check_latest=False)

In [ ]:
sections_with_nothing_before = []
slicebefore_name = 's023'
sections_with_nothing_before.append(slicebefore_name)

In [ ]:
sections_with_nothing_before

In [ ]:
lcm_directory = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_V1_MAPseq/BRAC8198.6c/LCM_registration') 
json_vis = lcm_directory/'reference/191123_visualign.json'#where registration file is saved
#savingpath = ('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/allenccf/allen_ccf_coord') #where you want to save
#below is path where allen annotation nrrd file is
allen_anno_path = '/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/reference/annotation_25.nrrd'

In [ ]:
lrf.convert_images(lcm_aligned_dir= json_vis,use_slurm=True, slurm_folder='/camp/home/turnerb/slurm_logs')

In [ ]:
OB_first = 'no' #direction of slicing. If first few sections are OB, type 'yes'
#sections where there is a missing section before -> includes first slice
add_z = get_z_value(lcm_dir=lcm_directory, OB_first = OB_first)


In [ ]:
ROI_vol = lrf.get_roi_vol(lcm_dir=lcm_directory, add_z=add_z, allen_anno_path=allen_anno_path, z_moving_towards_OB='no')

In [ ]:
ROI_vol =pd.read_pickle(lcm_directory/'ROI_vol.pkl', )

In [ ]:
lrf.combine_tubes(lcm_dir=str(lcm_directory), ROI_vol_path=str(lcm_directory/'ROI_vol.pkl'), use_slurm=True, slurm_folder='/camp/home/turnerb/slurm_logs')

In [ ]:
lcm_dir = '/camp/lab/znamenskiyp/home/shared/projects/turnerb_V1_MAPseq/BRAC8198.6c/LCM_registration'
lrf.get_euclidean_distance(lcm_dir, use_slurm=True, slurm_folder='/camp/home/turnerb/slurm_logs')

In [ ]:
slice = 's056'
directory = '/camp/lab/znamenskiyp/home/shared/projects/turnerb_V1_MAPseq/BRAC8198.6c/LCM_registration'
saving_path = pathlib.Path(directory)/'allenccf/allen_ccf_coord'
    
slicenum = int(slice[1:4])
slice_before= slicenum+1
if slice_before >9:
    slicebefore_name = f's0{slice_before}'
if slice_before<10:
    slicebefore_name = f's00{slice_before}' 
[x1a, y1a, z1a, one1] = np.load(saving_path/f'allen_ccf_converted_{slice}.npy')
[x2a, y2a, z2a, one2] = np.load(saving_path/f'allen_ccf_converted_{slicebefore_name}.npy')
point_z = x1a.flatten()
points_xy = np.column_stack((z1a.flatten(), y1a.flatten()))
# Stack x1 and y1 to create a single array for [x1, y1]
points_xy1 = np.column_stack((z2a.flatten(), y2a.flatten()))
point_z2 = x2a.flatten()
# Find the closest point in x1, y1 for each point in x, y
z_dist_points = []
for i, point in enumerate(points_xy):
    distances = np.linalg.norm(points_xy1 - point, axis=1)
    closest_index = np.argmin(distances)
    z_distance = point_z2[closest_index] - point_z[i]
    z_dist_points.append(z_distance)

In [ ]:
x1a.shape

In [ ]:
z_dist_points.shape

In [ ]:
z_dist_points= np.array(z_dist_points)

In [ ]:
z_dist_points =z_dist_points.reshape(158, 2)

In [ ]:
for i, r in ROI_vol.iterrows():
    unique = r.unique_regions
    for x in unique:
        if x < 1:
            if x > 0:
                print(x, i, r['tube'])
        

In [ ]:
lcm_dir = lcm_directory
roi_path = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_V1_MAPseq/BRAC8198.6c/LCM_registration/rois/')
image_reverse_of_cutting_side = 'yes'
z_to_add = add_z
region = 'S042_TUBE123.png'
slice_name = f's{region[1:4]}'
tube = region[5:len(region)].split('TUBE', 1)[1]
tube =tube[:-4]
[xa, ya, za, one] = np.load(lcm_dir/f'allenccf/allen_ccf_coord/allen_ccf_converted_{slice_name}.npy')
roi = plt.imread(roi_path / f'{region}')
allencoord_roiya = roi*ya
allencoord_roiza = roi*za
#use shoelace formula to define area of polygon given xy coordinates then calculate volume of each LCM roi
calcz = allencoord_roiza[allencoord_roiza!= 0]
calcy = allencoord_roiya[allencoord_roiya!= 0]
area_roi = 0.5*np.abs(np.dot(calcz,np.roll(calcy,1))-np.dot(calcy,np.roll(calcz,1)))
z_to_add = add_z.loc[add_z['slice'] == slice_name, 'amountz'].iloc[0]
if image_reverse_of_cutting_side == 'yes':
    vol_roi = area_roi*z_to_add
else:
    vol_roi = area_roi*-z_to_add
#convert the x, y, z coordinates to pixel
pixcoord = []
for i, axis in enumerate([xa, ya, za]):
    pixel = np.array(np.round(axis/25), dtype=int)
    pixel[pixel <0] = 0
    pixel[pixel >= annotation.shape[i]] = 0
    pixcoord.append(pixel)

In [ ]:
registered_slice = np.zeros(xa.shape, dtype = annotation.dtype)
a2=annotation[pixcoord[0].flatten(),
            pixcoord[1].flatten(),
            pixcoord[2].flatten()].reshape(registered_slice.shape)
ROI_anno = a2*roi

In [ ]:
np.unique(ROI_anno)

In [ ]:
lrf.combine_tubes(lcm_dir=str(lcm_directory), ROI_vol_path=str(lcm_directory/'ROI_vol.pkl'), use_slurm=True, slurm_folder='/camp/home/turnerb/slurm_logs')

In [ ]:
#load annotation
allen_anno = nrrd.read(allen_anno_path)
allen_anno = np.array(allen_anno)
annotation = allen_anno[0]

In [ ]:
#generate list of unique id regions in all samples
for r, tube in final_pix['tube'].iteritems():
    if r ==0 :
        array = final_pix.loc[r, 'combined_pix']
    if r > 0:
        next_array = final_pix.loc[r, 'combined_pix']
        array = np.concatenate((array, next_array), axis=None)

all_regions= np.unique(array)
region_col =  all_regions[all_regions!= 0]
final_pix.tube = final_pix.tube.astype(float)
finalpix1 =final_pix.sort_values('tube').reset_index()
np.save(lcm_directory/'region_col.npy', region_col)
finalpix1.to_pickle(lcm_directory/"finalpix1.pkl")
final_pix.to_pickle(lcm_directory/"finalpix.pkl")

In [ ]:
#load annotation
allen_anno = nrrd.read(allen_anno_path)
allen_anno = np.array(allen_anno)
annotation = allen_anno[0]

In [ ]:
roi_path = pathlib.Path(directory).parents[1]/'rois'
ROI_vol=pd.DataFrame()
for region in os.listdir(roi_path):
    if region.startswith("s0"):
        slice_name = region[0:4]
        tube = region[5:len(region)].split('TUBE', 1)[1]
        tube =tube[:-4]
        [xa, ya, za, one] = np.load(f'allenccf/allen_ccf_coord/allen_ccf_converted_{slice_name}.npy')
        roi = plt.imread('rois/%s' %region)
        allencoord_roiya = roi*ya
        allencoord_roiza = roi*za
#use shoelace formula to define area of polygon given xy coordinates then calculate volume of each LCM roi
        calcz = allencoord_roiza[allencoord_roiza!= 0]
        calcy = allencoord_roiya[allencoord_roiya!= 0]
        area_roi = 0.5*np.abs(np.dot(calcz,np.roll(calcy,1))-np.dot(calcy,np.roll(calcz,1)))
        z_to_add = add_z.loc[add_z['slice'] == slice_name, 'amountz'].iloc[0]
        vol_roi = area_roi*-z_to_add
#convert the x, y, z coordinates to pixel
        pixcoord = []
        for i, axis in enumerate([xa, ya, za]):
            pixel = np.array(np.round(axis/25), dtype=int)
            pixel[pixel <0] = 0
            pixel[pixel >= annotation.shape[i]] = 0
            pixcoord.append(pixel)

# use annotation.json to convert each pixel to region id

        registered_slice = np.zeros(xa.shape, dtype = annotation.dtype)
        a2=annotation[pixcoord[0].flatten(),
                      pixcoord[1].flatten(),
                      pixcoord[2].flatten()].reshape(registered_slice.shape)
        ROI_anno = a2*roi
#iterate image by z slices, each additional z, annotate then add to list
        slices = -round(z_to_add/25)
        for x in range(slices):
            if x >0:
                newz = pixcoord[0]-x #changed from plus to minus as going backwards
                slice = annotation[newz.flatten(), pixcoord[1].flatten(), pixcoord[2].flatten()].reshape(registered_slice.shape)
                ROI_anno_add = slice*roi
                ROI_anno = np.append(ROI_anno, ROI_anno_add)
    
        unique, counts = np.unique(ROI_anno, return_counts=True)
        region_vol = (counts/sum(counts))*vol_roi
        ROI_vol= ROI_vol.append({'slice': slice_name, 'tube': tube, 'z_added': z_to_add, 'vol (um3)': vol_roi, 'region_pix': ROI_anno, 'unique_regions': unique[1:], 'region_vol (um3)': region_vol[1:]},ignore_index=True)



In [ ]:
ROI_saving_path = pathlib.Path(directory).parents[1]/'allenccf'
ROI_vol.to_pickle(ROI_saving_path/"ROI_vol.pkl")


In [ ]:
#combine volumes for LCM
finalpix = pd.DataFrame(columns=['tube', 'combined_pix', 'vol (um3)'], dtype=int)
result = ROI_vol.groupby(['tube']).agg(', '.join).reset_index()

In [ ]:
for row, tube in result['tube'].iteritems():
    newdf = ROI_vol[ROI_vol['tube']==tube].reset_index()
    #for count, value in enumerate(newdf):
    for r, t in newdf['tube'].iteritems():
        if r ==0 :
            array = newdf.loc[r, 'region_pix']
            vol = newdf.loc[r, 'vol (um3)']
        if r > 0:
            nextarray = newdf.loc[r, 'region_pix']
            vol = vol + newdf.loc[r, 'vol (um3)']
            array = np.concatenate((array, nextarray), axis=None)
    final_pix= final_pix.append({'tube': tube, 'combined_pix': array, 'vol (um3)': vol},ignore_index=True)

In [ ]:
#generate list of unique id regions in all samples
for r, tube in final_pix['tube'].iteritems():
    if r ==0 :
        array = final_pix.loc[r, 'combined_pix']
    if r > 0:
        next_array = final_pix.loc[r, 'combined_pix']
    array = np.concatenate((array, next_array), axis=None)
all_regions= np.unique(array)
region_col =  all_regions[all_regions!= 0]
#calculate region volume in each tube, then create a heatmap of regions coloured according to region
final_pix.tube = final_pix.tube.astype(float)
finalpix1 =final_pix.sort_values('tube').reset_index()
# np.save('region_col.npy', region_col)
# finalpix1.to_pickle("finalpix1.pkl")
# final_pix.topickle("finalpix.pkl")

In [ ]:
lcm_directory

In [ ]:
finalpix1= pd.read_pickle(lcm_directory/"finalpix.pkl")

In [ ]:
finalpix1

In [ ]:
regioncol = np.load(lcm_directory/"region_col.npy")

In [ ]:
#now generate empty table for the acronymns of all areas based on allen ccf
acronymncol = []
for id in regioncol:
    if bg_atlas.structures[id]['acronym'][-1].isnumeric():
        newid = bg_atlas.structures[id]["structure_id_path"][-2] #moving one level up the hierarchy if cortical layer
    elif bg_atlas.structures[id]['acronym'][-2:]== '6a' or bg_atlas.structures[id]['acronym'][-2:]== '6b':
        newid = bg_atlas.structures[id]["structure_id_path"][-2] #moving one level up the hierarchy if layer 6a/6b
    else: newid = id
    acronymn = bg_atlas.structures[newid]['acronym']
    acronymncol.append(acronymn)
acronymncol= np.unique(acronymncol).tolist()
region_table = pd.DataFrame(columns = acronymncol, dtype =int)

In [ ]:
#need to generate reference table to convert id's into higher bit of hierarchy.

for row, tube in finalpix1['tube'].iteritems():
    regions = finalpix1.loc[row, 'combined_pix']
    unique, counts = np.unique(regions, return_counts=True)
    region_area = (counts/sum(counts))*(finalpix1.loc[row, 'vol (um3)'])
    regions = unique[1:]
    region_area = region_area[1:]
    values = regions, region_area
    region_table.at[row, 'sample'] = tube
    index = -1
    if regions.size != 0:
        for id in np.nditer(regions):
            index += 1
            if bg_atlas.structures[id]['acronym'][-1].isnumeric():
                newid = bg_atlas.structures[id]["structure_id_path"][-2] #moving one level up the hierarchy if cortical layer
            elif bg_atlas.structures[id]['acronym'][-2:]== '6a' or bg_atlas.structures[id]['acronym'][-2:]== '6b':
                newid = bg_atlas.structures[id]["structure_id_path"][-2] #moving one level up the hierarchy if layer 6a/6b
            else: newid = id
            acronym = bg_atlas.structures[newid]['acronym']
            region_table.at[row, acronym]= region_area[index]



In [ ]:
contra_samples= [27, 45, 46, 80, 90, 105, 118, 125]

In [ ]:
region_table

In [ ]:
region_tab_contra = region_table
#take areas in samples of contralateral hemisphere, and re-label as belonging to contra

for i, row in region_table.iterrows():
    if region_table['sample'].iloc[i] in contra_samples:
        for col in region_table.columns:
            if col != 'sample'and col.startswith('Contra')==False and np.isnan(region_table[col].iloc[i])== False:
                newcol = 'Contra-' + col
                if newcol not in region_tab_contra:
                    region_tab_contra[newcol]=0
                region_tab_contra[newcol].iloc[i] = region_table[col].iloc[i]
                region_tab_contra[col].iloc[i] =0
            
nozero = region_table.fillna(0)

In [ ]:
nozero

In [ ]:
nozero.to_csv('3d_areas.csv',index=False)

In [ ]:
region_table

In [ ]:
id = 269
if bg_atlas.structures[id]['acronym'][-2:]== '6a' or bg_atlas.structures[id]['acronym'][-2:]== '6b':
    print('yes')

In [ ]:
bg_atlas.structures[251]['acronym'][-2:]

In [ ]:
df= nozero.reindex(sorted(nozero.columns), axis=1)
df= df.drop(['sample'], axis=1)

fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(df, norm=LogNorm(), cbar_kws={'label': 'Vol (um3)'})
ax.set(xlabel="Brain region",
      ylabel="Sample")

plt.savefig(f'{lcm_directory}/region_heatmap.png')
plt.show()

In [ ]:
bg_atlas.structures[bg_atlas.structures[78]["structure_id_path"][-2]]['acronym']

In [ ]:
finalpix.tube = finalpix.tube.astype(float)
finalpix1 =finalpix.sort_values('tube').reset_index()